<a href="https://colab.research.google.com/github/BeleRohit/Fine_Tuning/blob/main/fine_tuning_deepseek_r1_maths.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




Packages Used will be:

- `unsloth`: Efficient fine-tuning and inference for LLMs — Specifically we will be using:
    - `FastLanguageModel` module to optimize inference & fine-tuning
    - `get_peft_model` to enable LoRa (Low-Rank Adaptation) fine-tuning
- `peft`: Supports LoRA-based fine-tuning for large models.
- Different Hugging Face modules:
    - `transformers` from HuggingFace to work with our fine-tuning data and handle different model tasks
    - `trl` Transformer Reinforcement Learning from HuggingFace which allows for supervised fine-tuning of the model — we will use the `SFFTrainer` wrapper
    - `datasets` to fetch reasoning datasets from the Hugging Face Hub
- `torch`: Deep learning framework used for training
- `wandb`: Provides access to weights and biases for tracking our fine-tuning experiment

In [ ]:
from google.colab import userdata
hugging_face_token = userdata.get("Hugging_Face_Token")
wnb_token = userdata.get("wnb")


## Install relevant packages

In [ ]:
%%capture

!pip install unsloth # install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
print('done')

## Import all relevant packages throughout this walkthrough

In [ ]:
# Modules for fine-tuning
from unsloth import FastLanguageModel
import torch # Import PyTorch
from trl import SFTTrainer # Trainer for supervised fine-tuning (SFT)
from unsloth import is_bfloat16_supported # Checks if the hardware supports bfloat16 precision
# Hugging Face modules
from huggingface_hub import login # Lets you login to API
from transformers import TrainingArguments # Defines training hyperparameters
from datasets import load_dataset # Lets you load fine-tuning datasets
# Import weights and biases
import wandb


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Create API keys and login to Hugging Face and Weights and Biases

In [ ]:
# Initialize Hugging Face & WnB tokens
from google.colab import userdata
hugging_face_token = userdata.get("Hugging_Face_Token")
wnb_token = userdata.get("wnb")


# Login to Hugging Face
login(hugging_face_token) # from huggingface_hub import login

# Login to WnB
wandb.login(key=wnb_token) # import wandb
run = wandb.init(
    project='Fine-tune-DeepSeek',
    job_type="training",
    anonymous="allow"
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: braintalks603 (braintalks603-lovely-professional-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


## Loading DeepSeek R1 and the Tokenizer

**What are we doing in this step?**

In this step, we **load the DeepSeek R1 model and its tokenizer** using `FastLanguageModel.from_pretrained()`. We also **configure key parameters** for efficient inference and fine-tuning. We will be using a distilled 8B version of R1 for faster computation.  

**Key parameters explained**
```py
max_seq_length = 2048  # Define the maximum sequence length a model can handle (i.e., number of tokens per input)
dtype = None  # Default data type (usually auto-detected)
load_in_4bit = True  # Enables 4-bit quantization – a memory-saving optimization
```

**Intuition behind 4-bit quantization**

Imagine compressing a **high-resolution image** to a smaller size—**it takes up less space but still looks good enough**. Similarly, **4-bit quantization reduces the precision of model weights**, making the model **smaller and faster while keeping most of its accuracy**. Instead of storing precise **32-bit or 16-bit numbers**, we compress them into **4-bit values**. This allows **large language models to run efficiently on consumer GPUs** without needing massive amounts of memory.

In [ ]:
# Set parameters
max_seq_length = 2048 # Define the maximum sequence length a model can handle (i.e. how many tokens can be processed at once)
dtype = None # Set to default
load_in_4bit = True # Enables 4 bit quantization — a memory saving optimization

# Load the DeepSeek R1 model and tokenizer using unsloth — imported using: from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",  # Load the pre-trained DeepSeek R1 model (8B parameter version)
    max_seq_length=max_seq_length, # Ensure the model can process up to 2048 tokens at once
    dtype=dtype, # Use the default data type (e.g., FP16 or BF16 depending on hardware support)
    load_in_4bit=load_in_4bit, # Load the model in 4-bit quantization to save memory
    token=hugging_face_token, # Use hugging face token
)

==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.9k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

## Testing DeepSeek R1 on a Mathematical Question


### Defining a system prompt
To create a prompt style for the model, we will define a system prompt and include placeholders for the question and response generation. The prompt will guide the model to think step-by-step and provide a logical, accurate response.

In [ ]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a mathematical expert with advanced knowledge in data analysis, statistical modeling, and algorithmic problem-solving.
Please analyze and solve the following mathematical dataset-related problem.

### Question:
{}

### Response:
<think>{}"""


### Running inference on the model

In this step, we **test the DeepSeek R1 model** by providing a **medical question** and generating a response.  
The process involves the following steps:

1. **Define a test question** related to a medical case.
2. **Format the question using the structured prompt (`prompt_style`)** to ensure the model follows a logical reasoning process.
3. **Tokenize the input and move it to the GPU (`cuda`)** for faster inference.
4. **Generate a response using the model**, specifying key parameters like `max_new_tokens=1200` (limits response length).
5. **Decode the output tokens back into text** to obtain the final readable answer.

In [ ]:
# Creating a test medical question for inference
question = """A dataset contains records of a company’s monthly revenue over 10 years. The revenue follows a linear trend
              with seasonal fluctuations. Given this pattern, what would a time series decomposition most likely reveal
              about the trend component and seasonal component of the dataset?"""


# Enable optimized inference mode for Unsloth models (improves speed and efficiency)
FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!

# Format the question using the structured prompt (`prompt_style`) and tokenize it
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")  # Convert input to PyTorch tensor & move to GPU

# Generate a response using the model
outputs = model.generate(
    input_ids=inputs.input_ids, # Tokenized input question
    attention_mask=inputs.attention_mask, # Attention mask to handle padding
    max_new_tokens=1200, # Limit response length to 1200 tokens (to prevent excessive output)
    use_cache=True, # Enable caching for faster inference
)

# Decode the generated output tokens into human-readable text
response = tokenizer.batch_decode(outputs)

# Extract and print only the relevant response part (after "### Response:")
print(response[0].split("### Response:")[1])

>**Before starting fine-tuning — why are we fine-tuning in the first place?**
>
> Even without fine-tuning, our model successfully generated a chain of thought and provided reasoning before delivering the final answer. The reasoning process is encapsulated within the `<think>` `</think>` tags. So, why do we still need fine-tuning? The reasoning process, while detailed, was long-winded and not concise. Additionally, we want the final answer to be consistent in a certain style.



## Fine-tuning step by step

### Step 1 — Update the system prompt
We will slightly change the prompt style for processing the dataset by adding the third placeholder for the complex chain of thought column. `</think>`

In [ ]:
# Updated training prompt style to add </think> tag
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:

You are a math expert with advanced knowledge in mathematical reasoning, problem-solving, and quantitative analysis.
Your task is to answer questions with a strong foundation in mathematical principles, ensuring precision, logical consistency, and rigor.
Please Answer the following math question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""


### Step 2 — Download the fine-tuning dataset and format it for fine-tuning
we'll be using math dataset from Openthoughts.

In [ ]:
# Download the dataset using Hugging Face — function imported using from datasets import load_dataset
dataset = load_dataset("open-r1/OpenThoughts-114k-math", split = "train[0:500]",trust_remote_code=True) # Keep only first 500 rows
dataset

train-00000-of-00005.parquet:   0%|          | 0.00/199M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/201M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/203M [00:00<?, ?B/s]

train-00003-of-00005.parquet:   0%|          | 0.00/200M [00:00<?, ?B/s]

train-00004-of-00005.parquet:   0%|          | 0.00/176M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/89120 [00:00<?, ? examples/s]

Dataset({
    features: ['source', 'problem', 'solution', 'messages', 'system', 'conversations', 'generated_token_count', 'correct'],
    num_rows: 500
})

In [ ]:
# Show an entry from the dataset
import pprint
pprint.pprint(dataset[1])
#type(dataset[1])

{'conversations': [{'from': 'user',
                    'value': 'Return your final response within \\boxed{}. '
                             'Given real numbers \\( a, b, c \\) and a '
                             'positive number \\( \\lambda \\) such that the '
                             'polynomial \\( f(x) = x^3 + a x^2 + b x + c \\) '
                             'has three real roots \\( x_1, x_2, x_3 \\), and '
                             'the conditions \\( x_2 - x_1 = \\lambda \\) and '
                             '\\( x_3 > \\frac{1}{2}(x_1 + x_2) \\) are '
                             'satisfied, find the maximum value of \\( '
                             '\\frac{2 a^3 + 27 c - 9 a b}{\\lambda^3} \\).'},
                   {'from': 'assistant',
                    'value': '<|begin_of_thought|>\n'
                             '\n'
                             "Okay, let's try to tackle this problem step by "
                             'step. The problem is about find

>**Next step is to structure the fine-tuning dataset according to train prompt style—why?**
>
> - Each question is paired with chain-of-thought reasoning and the final response.
> - Ensures every training example follows a consistent pattern.
> - Prevents the model from continuing beyond the expected response lengt by adding the EOS token.

In [ ]:
# We need to format the dataset to fit our prompt training style
EOS_TOKEN = tokenizer.eos_token  # Define EOS_TOKEN which the model when to stop generating text during training
EOS_TOKEN

'<｜end▁of▁sentence｜>'

In [ ]:
# Define formatting prompt function
def formatting_prompts_func(examples):  # Takes a batch of dataset examples as input
    inputs = examples["problem"]       # Extracts the medical question from the dataset
    cots = examples["system"]      # Extracts the chain-of-thought reasoning (logical step-by-step explanation)
    outputs = examples["solution"]      # Extracts the final model-generated response (answer)

    texts = []  # Initializes an empty list to store the formatted prompts

    # Iterate over the dataset, formatting each question, reasoning step, and response
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN  # Insert values into prompt template & append EOS token
        texts.append(text)  # Add the formatted text to the list

    return {
        "text": texts,  # Return the newly formatted dataset with a "text" column containing structured prompts
    }

In [ ]:
# Update dataset formatting
dataset_finetune = dataset.map(formatting_prompts_func, batched = True)
dataset_finetune["text"][0]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

"Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request. \nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\n\nYou are a math expert with advanced knowledge in mathematical reasoning, problem-solving, and quantitative analysis.\nYour task is to answer questions with a strong foundation in mathematical principles, ensuring precision, logical consistency, and rigor. \nPlease Answer the following math question.\n\n### Question:\nLet \\( a, b, c \\) be positive real numbers. Prove that\n\n$$\n\\frac{1}{a(1+b)}+\\frac{1}{b(1+c)}+\\frac{1}{c(1+a)} \\geq \\frac{3}{1+abc},\n$$\n\nand that equality occurs if and only if \\( a = b = c = 1 \\).\n\n### Response:\n<think>\nYour role as an assistant involves thoroughly exploring questions through a systematic long thinking process before 

### Step 3 — Setting up the model using LoRA




In [ ]:
# Apply LoRA (Low-Rank Adaptation) fine-tuning to the model
model_lora = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank: Determines the size of the trainable adapters (higher = more parameters, lower = more efficiency)
    target_modules=[  # List of transformer layers where LoRA adapters will be applied
        "q_proj",   # Query projection in the self-attention mechanism
        "k_proj",   # Key projection in the self-attention mechanism
        "v_proj",   # Value projection in the self-attention mechanism
        "o_proj",   # Output projection from the attention layer
        "gate_proj",  # Used in feed-forward layers (MLP)
        "up_proj",    # Part of the transformer’s feed-forward network (FFN)
        "down_proj",  # Another part of the transformer’s FFN
    ],
    lora_alpha=16,  # Scaling factor for LoRA updates (higher values allow more influence from LoRA layers)
    lora_dropout=0,  # Dropout rate for LoRA layers (0 means no dropout, full retention of information)
    bias="none",  # Specifies whether LoRA layers should learn bias terms (setting to "none" saves memory)
    use_gradient_checkpointing="unsloth",  # Saves memory by recomputing activations instead of storing them (recommended for long-context fine-tuning)
    random_state=3407,  # Sets a seed for reproducibility, ensuring the same fine-tuning behavior across runs
    use_rslora=False,  # Whether to use Rank-Stabilized LoRA (disabled here, meaning fixed-rank LoRA is used)
    loftq_config=None,  # Low-bit Fine-Tuning Quantization (LoFTQ) is disabled in this configuration
)

Unsloth 2025.1.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Now, we initialize `SFTTrainer`, a supervised fine-tuning trainer from `trl` (Transformer Reinforcement Learning), to fine-tune our model efficiently on a dataset.

In [ ]:
# Initialize the fine-tuning trainer — Imported using from trl import SFTTrainer
trainer = SFTTrainer(
    model=model_lora,  # The model to be fine-tuned
    tokenizer=tokenizer,  # Tokenizer to process text inputs
    train_dataset=dataset_finetune,  # Dataset used for training
    dataset_text_field="text",  # Specifies which field in the dataset contains training text
    max_seq_length=max_seq_length,  # Defines the maximum sequence length for inputs
    dataset_num_proc=2,  # Uses 2 CPU threads to speed up data preprocessing

    # Define training arguments
    args=TrainingArguments(
        per_device_train_batch_size=2,  # Number of examples processed per device (GPU) at a time
        gradient_accumulation_steps=4,  # Accumulate gradients over 4 steps before updating weights
        num_train_epochs=1, # Full fine-tuning run
        warmup_steps=5,  # Gradually increases learning rate for the first 5 steps
        max_steps=60,  # Limits training to 60 steps (useful for debugging; increase for full fine-tuning)
        learning_rate=2e-4,  # Learning rate for weight updates (tuned for LoRA fine-tuning)
        fp16=not is_bfloat16_supported(),  # Use FP16 (if BF16 is not supported) to speed up training
        bf16=is_bfloat16_supported(),  # Use BF16 if supported (better numerical stability on newer GPUs)
        logging_steps=10,  # Logs training progress every 10 steps
        optim="adamw_8bit",  # Uses memory-efficient AdamW optimizer in 8-bit mode
        weight_decay=0.01,  # Regularization to prevent overfitting
        lr_scheduler_type="linear",  # Uses a linear learning rate schedule
        seed=3407,  # Sets a fixed seed for reproducibility
        output_dir="outputs",  # Directory where fine-tuned model checkpoints will be saved
    ),
)


Map (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

## Step 4 — Model training!

This should take around 30 to 40 minutes — we can then check out our training results on Weights and Biases

In [ ]:
# Start the fine-tuning process
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,0.860200
20,0.787300
30,0.676900
40,0.698600
50,0.677200
60,0.671300


In [ ]:
# Save the fine-tuned model
wandb.finish()

train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▅▁▆▄▂
train/learning_rate,█▇▅▄▂▁
train/loss,█▅▁▂▁▁
total_flos,2.1616885624602624e+16
train/epoch,0.96
train/global_step,60
train/grad_norm,0.15691
train/learning_rate,0
train/loss,0.6713


## Step 5 — Run model inference after fine-tuning

In [ ]:
question = """Solve for \( x \) in the equation:
\[
\frac{2x + 3}{5} + \frac{x - 2}{3} = \frac{4x + 1}{6}
\]
"""

# Load the inference model using FastLanguageModel (Unsloth optimizes for speed)
FastLanguageModel.for_inference(model_lora)  # Unsloth has 2x faster inference!

# Tokenize the input question with a specific prompt format and move it to the GPU
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response using LoRA fine-tuned model with specific parameters
outputs = model_lora.generate(
    input_ids=inputs.input_ids,          # Tokenized input IDs
    attention_mask=inputs.attention_mask, # Attention mask for padding handling
    max_new_tokens=1200,                  # Maximum length for generated response
    use_cache=True,                        # Enable cache for efficient generation
)

# Decode the generated response from tokenized format to readable text
response = tokenizer.batch_decode(outputs)

# Extract and print only the model's response part after "### Response:"
print(response[0].split("### Response:")[1])


<think>
To solve the equation:
\[
\frac{2x + 3}{5} + \frac{x - 2}{3} = \frac{4x + 1}{6}
\]

1. **Find a common denominator** for the fractions. The denominators are 5, 3, and 6. The least common multiple (LCM) of these denominators is 30.

2. **Rewrite each fraction** with the common denominator 30:
   \[
   \frac{2x + 3}{5} = \frac{6(2x + 3)}{30}, \quad \frac{x - 2}{3} = \frac{10(x - 2)}{30}, \quad \frac{4x + 1}{6} = \frac{5(4x + 1)}{30}
   \]

3. **Combine the fractions**:
   \[
   \frac{6(2x + 3) + 10(x - 2)}{30} = \frac{5(4x + 1)}{30}
   \]

4. **Simplify the numerators**:
   \[
   6(2x + 3) + 10(x - 2) = 12x + 18 + 10x - 20 = 22x - 2
   \]
   \[
   5(4x + 1) = 20x + 5
   \]

5. **Set the equations equal**:
   \[
   \frac{22x - 2}{30} = \frac{20x + 5}{30}
   \]

6. **Solve for \( x \)**:
   \[
   22x - 2 = 20x + 5
   \]
   \[
   22x - 20x = 5 + 2
   \]
   \[
   2x = 7
   \]
   \[
   x = \frac{7}{2} = 3.5
   \]

### Conclusion:
\[
\boxed{x = 3.5}
\]<｜end▁of▁sentence｜>


In [ ]:
#from sympy.parsing.latex import parse_latex
## JEE Advanced 2024 question


question = """A student appears for a quiz consisting of only true-false type questions and answers all the questions.
The student knows the answers of some questions and guesses the answers for the remaining
questions. Whenever the student knows the answer of a question, he gives the correct answer.
Assume that the probability of the student giving the correct answer for a question, given that he has
guessed it, is 1/2 Also assume that the probability of the answer for a question being guessed, given
that the student’s answer is correct, is 1/6 Then the probability that the student knows the answer of
a randomly chosen question is """

# Tokenize the input question with a specific prompt format and move it to the GPU
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response using LoRA fine-tuned model with specific parameters
outputs = model_lora.generate(
    input_ids=inputs.input_ids,          # Tokenized input IDs
    attention_mask=inputs.attention_mask, # Attention mask for padding handling
    max_new_tokens=1200,                  # Maximum length for generated response
    use_cache=True,                        # Enable cache for efficient generation
)

# Decode the generated response from tokenized format to readable text
response = tokenizer.batch_decode(outputs)

# Extract and print only the model's response part after "### Response:"
print(response[0].split("### Response:")[1])



<think>
1. **Identify the probability of the student correctly answering a question when he knows it**: 
   - Let \( P(A) \) be the probability that the student knows the answer to a question. 
   - Let \( P(C) \) be the probability that the student correctly answers a question.
   - Given that the student knows the answer, the probability of correctly answering is \( P(C | A) = 1 \).

2. **Calculate the probability of the student correctly answering a question by guessing**: 
   - Let \( P(G) \) be the probability that the student guesses a question.
   - Let \( P(C | G) \) be the probability of correctly answering a guessed question.
   - Given that the student guesses, the probability of correctly answering is \( P(C | G) = \frac{1}{2} \).

3. **Determine the probability that the student knows the answer**: 
   - We need to find \( P(A) \) given \( P(C) \):
   \[
   P(C) = P(A) \cdot P(C | A) + P(G) \cdot P(C | G)
   \]
   - Substitute \( P(C | A) = 1 \) and \( P(C | G) = \frac{1}{

In [ ]:
import re

raw_response = response[0].split("### Response:")[1]

# Remove LaTeX commands and braces
cleaned_response = re.sub(r'\\[a-zA-Z]+|[{}$]', '', raw_response)

# Convert LaTeX fractions like \frac{a}{b} to a/b
cleaned_response = re.sub(r'\\frac{(\d+)}{(\d+)}', r'\1/\2', cleaned_response)

print(cleaned_response)


<think>
1. **Identify the probability of the student correctly answering a question when he knows it**: 
   - Let \( P(A) \) be the probability that the student knows the answer to a question. 
   - Let \( P(C) \) be the probability that the student correctly answers a question.
   - Given that the student knows the answer, the probability of correctly answering is \( P(C | A) = 1 \).

2. **Calculate the probability of the student correctly answering a question by guessing**: 
   - Let \( P(G) \) be the probability that the student guesses a question.
   - Let \( P(C | G) \) be the probability of correctly answering a guessed question.
   - Given that the student guesses, the probability of correctly answering is \( P(C | G) = 12 \).

3. **Determine the probability that the student knows the answer**: 
   - We need to find \( P(A) \) given \( P(C) \):
   \[
   P(C) = P(A)  P(C | A) + P(G)  P(C | G)
   \]
   - Substitute \( P(C | A) = 1 \) and \( P(C | G) = 12 \):
   \[
   P(C) = P(A) 